In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt 
from scipy.optimize import curve_fit
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import rc

In [ ]:
"""
class MyRandom():
    
    def __init__(self, seed = 15, method='simple'):
        
        self.r = seed
        self.method = method
        
        if method=='simple':
            self.a = 57
            self.c = 1
            self.M = 265
        elif method == 'drand48':
            self.a = int('5DEECE66D',16)
            self.c = int('B',16)
            self.M = 2**48
        else:
            print('Generador no reconocido')
            
    def Random(self):
        
        r = (self.a*self.r + self.c)%self.M
        self.r = r
        
        return r/float(self.M)
    
    def TestMethod(self, Npoints, moment, seed_ = 32, method_ = 'simple'):
        
        rand = MyRandom(seed = seed_, method = method_)
        
        array = np.zeros(Npoints)
            
        for i in range(Npoints):
            array[i] = rand.Random()
            
        return np.sqrt(Npoints)* np.abs(  np.mean(array**moment) - 1./(1.+moment) )
"""

In [ ]:
x = np.random.rand(10)
print(x)

In [ ]:
matrix = np.random.rand(10,5)
print(matrix)

In [ ]:
U = np.random.uniform( 0,10, 10000 )
Pois = np.random.poisson(size=10000)
Normal = np.random.standard_normal(size = 10000)
print(len(U),len(Pois),len(Normal),len(x))
print(U[0],Pois[0],Normal[0])
print(U[-1],Pois[-1],Normal[-1])

In [ ]:
rand = MyRandom(seed = 6625, method='drand48')

In [ ]:
print(rand.Random())

In [ ]:
rand2 = np.random.rand(10)

Points = np.logspace(2,6,5)
print(rand2,Points)

In [ ]:
NMoments = 10

Moments = []
for k in range(NMoments):
    test = []
    for i in range(len(Points)):
        test.append(rand2.TestMethod(int(Points[i]),k+1,96,'drand48'))
    Moments.append(test)

In [ ]:
labelk = []
for i in range(NMoments):
    labelk.append(i+1)

In [ ]:
fig = plt.figure( figsize=(10,10) )
ax1 = fig.add_subplot(2,2,1)
for i in range(NMoments):
    ax1.plot(Points,Moments[i], label=r'$k=%.0f$' %(labelk[i]))
    
ax1.set_xscale('log')
ax1.set_xlabel('N Points', fontsize=15)
ax1.set_ylabel('k-moment value', fontsize=15)
ax1.legend()

**Con Numpy**

In [ ]:
x = np.random.rand(10,10000)
print(x)

In [ ]:
matrix = np.random.rand(2,4)
print(matrix)

In [ ]:
U = np.random.uniform( -5,5, 10000 )
Pois = np.random.poisson(size=10000)
Normal = np.random.standard_normal(size = 10000)
print(len(U),len(Pois),len(Normal),len(x))

In [ ]:
fig = plt.figure( figsize=(10,10) )

ax1 = fig.add_subplot(2,2,1)
ax2 = fig.add_subplot(2,2,2)
ax3 = fig.add_subplot(2,2,3)
    ax1.plot(,U[i], label=r'$k=%.0f$' %(labelk[i]))
    ax2.plot(Points,Pois[i], label=r'$k=%.0f$' %(labelk[i]))   
    ax3.plot(Points,Normal[i], label=r'$k=%.0f$' %(labelk[i]))

ax1.set_xscale('log')
ax1.set_xlabel('N Points', fontsize=15)
ax1.set_ylabel('k-moment value', fontsize=15)
ax1.legend(loc=0)

ax2.set_xscale('log')
ax2.set_xlabel('N Points', fontsize=15)
ax2.set_ylabel('k-moment value', fontsize=15)
ax2.legend(loc=1)

ax3.set_xscale('log')
ax3.set_xlabel('N Points', fontsize=15)
ax3.set_ylabel('k-moment value', fontsize=15)
ax3.legend(loc=2)